## Utils

Conté el codi per descarregar i carregar el model GPT-2 dels repositoris oficials d'OpenAI. A posteriori carregarem els seus pesos, tokenitzador i hiperparàmetre.

In [3]:
import json
import os
import re

import numpy as np
import requests
import tensorflow as tf
from tqdm import tqdm

from modules.encoder import get_encoder